In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.sparse import csr_matrix

In [2]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [3]:
df_te = pd.read_hdf('input_hdf/df_test.hdf5','table')
faron_te = pd.read_csv('input_hdf/faron_test.csv')
#df_te = pd.read_hdf('input_hdf/df_test.hdf5','table')

In [4]:
df_te = df_te.drop(['Id'], axis=1)
faron_te = faron_te.drop(['Id'], axis=1)
#df_te = df_te.drop(['Id'], axis=1)

In [5]:
df_te.replace(-999999, np.nan, inplace=True)

In [6]:
df_te.head()

,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_2,L0_S1_F31_4,L0_S1_F31_5,L0_S2_F33_1,L0_S2_F35_1,L0_S2_F35_2,L0_S2_F35_3,L0_S2_F35_4,...,L3_S50_F4243,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
faron_te.head()

,StartTime,magic1,magic2,magic3,magic4
0,792.719971,-2367494,-1,-2324187,-3232
1,1025.630005,1,-1,-2353739,-5943
2,671.919983,1,-2,-2357112,-17769
3,255.449997,2,-3,-2364005,-33948
4,743.380005,3,-2,-1856001,-132849


In [8]:
df_te_with_faron = pd.concat([df_te,faron_te], axis=1)

In [9]:
df_te_with_faron.head()

,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_2,L0_S1_F31_4,L0_S1_F31_5,L0_S2_F33_1,L0_S2_F35_1,L0_S2_F35_2,L0_S2_F35_3,L0_S2_F35_4,...,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,StartTime,magic1,magic2,magic3,magic4
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,792.719971,-2367494,-1,-2324187,-3232
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1025.630005,1,-1,-2353739,-5943
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,671.919983,1,-2,-2357112,-17769
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,255.449997,2,-3,-2364005,-33948
4,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,743.380005,3,-2,-1856001,-132849


In [10]:
df_te_with_faron.to_hdf('df_te_with_faron.hdf5', 'table')

In [3]:
df_te_with_faron = pd.read_hdf('input_hdf/df_te_with_faron.hdf5', 'table')

In [4]:
df_te_with_faron.head()

,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_2,L0_S1_F31_4,L0_S1_F31_5,L0_S2_F33_1,L0_S2_F35_1,L0_S2_F35_2,L0_S2_F35_3,L0_S2_F35_4,...,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,StartTime,magic1,magic2,magic3,magic4
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,792.719971,-2367494,-1,-2324187,-3232
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1025.630005,1,-1,-2353739,-5943
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,671.919983,1,-2,-2357112,-17769
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,255.449997,2,-3,-2364005,-33948
4,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,743.380005,3,-2,-1856001,-132849


In [5]:
dtest = xgb.DMatrix(df_te_with_faron)

In [6]:
dtest.save_binary("test_with_faron.buffer")

In [ ]:
bst = xgb.Booster() #init model
bst.load_model("0001.model") # load data

from operator import itemgetter
get_importance(bst,df_tr.columns)

In [11]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             26G     0   26G   0% /dev
tmpfs           5.2G  8.7M  5.2G   1% /run
/dev/sda1       194G   98G   97G  51% /
tmpfs            26G     0   26G   0% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            26G     0   26G   0% /sys/fs/cgroup
tmpfs           5.2G   12K  5.2G   1% /run/user/1001
